<a href="https://colab.research.google.com/github/pranav-696/ML-Repo./blob/main/Deepfake_audio_Detectio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pywavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 25.4 MB/s eta 0:00:00


In [ ]:
!pip install flask
!pip install pyngrok
!pip install transformers

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile

# Define the path to your ZIP file in Google Drive
zip_file_path = '/content/drive/MyDrive/fake_audio.zip'

# Define the directory where you want to extract the files
extract_dir = '/content/dataset'

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Files extracted to:", extract_dir)

# Traverse the directory and list all file paths (optional)
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        file_path = os.path.join(root, file)
        print(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files extracted to: /content/dataset
/content/dataset/__MACOSX/._fake_audio
/content/dataset/__MACOSX/fake_audio/._readme.txt
/content/dataset/__MACOSX/fake_audio/._Key_fake_audios
/content/dataset/__MACOSX/fake_audio/._Fake_audios
/content/dataset/__MACOSX/fake_audio/._Original_audios
/content/dataset/__MACOSX/fake_audio/._.DS_Store
/content/dataset/__MACOSX/fake_audio/Fake_audios/._fake3_8.wav
/content/dataset/__MACOSX/fake_audio/Fake_audios/._fake2_10.wav
/content/dataset/__MACOSX/fake_audio/Fake_audios/._fake4_1.wav
/content/dataset/__MACOSX/fake_audio/Fake_audios/._fake1_9.wav
/content/dataset/__MACOSX/fake_audio/Fake_audios/._fake4_10.wav
/content/dataset/__MACOSX/fake_audio/Fake_audios/._fake2_8.wav
/content/dataset/__MACOSX/fake_audio/Fake_audios/._fake1_2.wav
/content/dataset/__MACOSX/fake_audio/Fake_audios/._fake4_7.wav
/content/dataset/__MACOSX/fak

In [ ]:
import os

# Define the directory where you unzipped the files
directory = '/content/dataset/fake_audio'

# Traverse the directory and list all file paths
for root, dirs, files in os.walk(directory):
    for file in files:
        file_path = os.path.join(root, file)
        print(file_path)

/content/dataset/fake_audio/.DS_Store
/content/dataset/fake_audio/readme.txt
/content/dataset/fake_audio/Fake_audios/fake1_7.wav
/content/dataset/fake_audio/Fake_audios/fake4_6.wav
/content/dataset/fake_audio/Fake_audios/fake3_2.wav
/content/dataset/fake_audio/Fake_audios/fake4_5.wav
/content/dataset/fake_audio/Fake_audios/fake3_8.wav
/content/dataset/fake_audio/Fake_audios/fake3_5.wav
/content/dataset/fake_audio/Fake_audios/fake1_10.wav
/content/dataset/fake_audio/Fake_audios/fake2_9.wav
/content/dataset/fake_audio/Fake_audios/fake1_4.wav
/content/dataset/fake_audio/Fake_audios/fake5_7.wav
/content/dataset/fake_audio/Fake_audios/fake2_7.wav
/content/dataset/fake_audio/Fake_audios/fake2_10.wav
/content/dataset/fake_audio/Fake_audios/fake1_3.wav
/content/dataset/fake_audio/Fake_audios/fake2_1.wav
/content/dataset/fake_audio/Fake_audios/fake2_2.wav
/content/dataset/fake_audio/Fake_audios/fake1_1.wav
/content/dataset/fake_audio/Fake_audios/fake4_3.wav
/content/dataset/fake_audio/Fake_audi

In [ ]:
# Model Training
import os
import numpy as np
import librosa
import pywt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Paths to the directories
fake_audio_dir = '/content/dataset/fake_audio/Fake_audios/'
key_dir = '/content/dataset/fake_audio/Key_fake_audios/'
original_audio_dir = '/content/dataset/fake_audio/Original_audios/'

# Function to load key from text file
def load_key(key_path):
    with open(key_path, 'r') as f:
        key = np.loadtxt(f, dtype=int)
    return key

# Function to recover original audio from fake using key
def recover_audio(fake_audio, key):
    coeffs = pywt.wavedec(fake_audio, wavelet='db10', level=4)
    Cm = np.copy(coeffs[0])

    key = np.clip(key, 0, len(Cm) - 1)

    Cm[key] = coeffs[0][key]
    recovered_audio = pywt.waverec([Cm] + coeffs[1:], wavelet='db10')
    return recovered_audio

# Function to extract MFCC features and pad them to a fixed length
def extract_mfcc(audio, sr, max_len=862):
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)

    if mfcc.shape[1] > max_len:
        mfcc = mfcc[:, :max_len]
    elif mfcc.shape[1] < max_len:
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    return mfcc

# Lists to hold features and labels
features_fake_recovered = []
features_original = []
labels = []

# Iterate through the fake audios
for i in range(1, 6):
    for j in range(1, 11):
        fake_audio_file = f'fake{i}_{j}.wav'

        # Load fake audio
        fake_audio_path = os.path.join(fake_audio_dir, fake_audio_file)
        fake_audio, sr = librosa.load(fake_audio_path, sr=None)

        # Load corresponding key
        key_file = f'key_fake{i}_{j}.txt'
        key_path = os.path.join(key_dir, key_file)
        key = load_key(key_path)

        # Recover the audio
        recovered_audio = recover_audio(fake_audio, key)

        # Extract features for both fake and recovered audio
        mfcc_fake = extract_mfcc(fake_audio, sr)
        mfcc_recovered = extract_mfcc(recovered_audio, sr)

        # Combine features
        combined_features = np.stack([mfcc_fake, mfcc_recovered], axis=-1)
        features_fake_recovered.append(combined_features)
        labels.append(1)

        # Load and process the corresponding original audio
        original_audio_file = f'speaker{i}_{j}.wav'
        original_audio_path = os.path.join(original_audio_dir, original_audio_file)
        original_audio, sr = librosa.load(original_audio_path, sr=None)

        mfcc_original = extract_mfcc(original_audio, sr)
        mfcc_original_expanded = np.stack([mfcc_original, mfcc_original], axis=-1)
        features_original.append(mfcc_original_expanded)
        labels.append(0)

# Convert lists to numpy arrays
features_fake_recovered = np.array(features_fake_recovered, dtype=np.float32)
features_original = np.array(features_original, dtype=np.float32)

# Combine features and labels
features = np.concatenate([features_fake_recovered, features_original], axis=0)
labels = np.array(labels)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define the CNN model
def build_detection_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build and compile the model
input_shape = X_train.shape[1:]
model = build_detection_model(input_shape)

# Early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('deepfake_audio_model.keras', save_best_only=True)

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=16,
                    callbacks=[early_stopping, model_checkpoint])

# Evaluate the model
y_pred = model.predict(X_val)
y_pred_classes = (y_pred > 0.5).astype(int)

print("Model training complete and saved as 'deepfake_audio_model.keras'")

Epoch 1/10
5/5 [==============================] - 12s 2s/step - loss: 24.9251 - accuracy: 0.4125 - val_loss: 4.5651 - val_accuracy: 0.4000
Epoch 2/10
5/5 [==============================] - 8s 2s/step - loss: 2.4098 - accuracy: 0.7125 - val_loss: 2.1699 - val_accuracy: 0.4500
Epoch 3/10
5/5 [==============================] - 10s 2s/step - loss: 2.2585 - accuracy: 0.7625 - val_loss: 0.8417 - val_accuracy: 0.8000
Epoch 4/10
5/5 [==============================] - 9s 2s/step - loss: 1.5165 - accuracy: 0.8125 - val_loss: 0.2751 - val_accuracy: 0.9000
Epoch 5/10
5/5 [==============================] - 8s 2s/step - loss: 0.1406 - accuracy: 0.9500 - val_loss: 0.1300 - val_accuracy: 0.9500
Epoch 6/10
5/5 [==============================] - 9s 2s/step - loss: 0.3959 - accuracy: 0.9500 - val_loss: 0.1891 - val_accuracy: 0.9000
Epoch 7/10
5/5 [==============================] - 7s 1s/step - loss: 0.0658 - accuracy: 0.9750 - val_loss: 0.2361 - val_accuracy: 0.8500
Epoch 8/10
1/1 [======================

In [ ]:
# Model testing and deploying
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
import tensorflow as tf
import numpy as np
import librosa

# Load the trained model
model = tf.keras.models.load_model('deepfake_audio_model.keras')

# Initialize the Flask app
app = Flask(__name__)

# Define the home route to serve the HTML form
@app.route('/')
def home():
    return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <link rel="stylesheet" type="text/css" href="/static/styles.css">
            <title>Deepfake Audio Detection</title>
        </head>
        <body>
          <div class="home">
            <h1>Upload an Audio File</h1>
            <form action="/predict" method="POST" enctype="multipart/form-data">
                <input type="file" name="file" accept="audio/*" required>
                <button class="btn" type="submit">Detect</button>
            </form>
        </body>
        </html>
    ''')

# Define the prediction route to handle audio file uploads
@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['file']

    # Extract features from the audio file
    audio, sr = librosa.load(file, sr=None)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfcc = np.pad(mfcc, pad_width=((0, 0), (0, 862 - mfcc.shape[1])), mode='constant')
    mfcc = np.expand_dims(mfcc, axis=-1)
    mfcc = np.repeat(mfcc, 2, axis=-1)
    mfcc = np.expand_dims(mfcc, axis=0)

    # Predict using the model
    prediction = model.predict(mfcc)
    probability_fake = prediction[0][0]
    probability_real = 1 - probability_fake

    output='''
            <style>
                body {
                    font-family: Arial, sans-serif;
                    background-color: #f4f4f4;
                    margin: 0;
                    padding: 0;
                }
                .container {
                    width: 80%;
                    margin: auto;
                    background-color: #fff;
                    padding: 20px;
                    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                    border-radius: 8px;
                    margin-top: 30px;
                }
                h2 {
                    color: #333;
                    text-align: center;
                    border-bottom: 2px solid #ddd;
                    padding-bottom: 10px;
                    margin-bottom: 20px;
                }
                .result {
                    padding: 20px;
                    border-radius: 8px;
                    background-color: #eaf0f7;
                    border: 1px solid #d0e3f0;
                    box-shadow: 0 0 8px rgba(0, 0, 0, 0.1);
                    text-align: center;
                    margin: 0 auto;
                    max-width: 300px;
                }
                .result h3 {
                    margin: 0;
                    color: #333;
                }
                .result .fake {
                    color: #ff4c4c;
                    font-weight: bold;
                }
                .result .real {
                    color: #4caf50;
                    font-weight: bold;
                }
            </style>
            <div class="container">
                <h2>Results:</h2>
                <div class="result">
                    <h3>Confidence Levels:</h3>
            '''

    output += f"<p class='fake'>Fake: {probability_real * 100:.2f}%</p>"
    output += f"<p class='real'>Real: {probability_fake * 100:.2f}%</p>"

    output += "</div>"
    output += "</div>"

    return output
#return jsonify({'result': result})

# Set up ngrok
authtoken = "2lVh1lhns8BObYcyJgVN7F2QA7t_6MdTZ4u6dgyA3GDkee2L3"  # Use your ngrok auth token
ngrok.set_auth_token(authtoken)
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# Run the Flask app
if __name__ == '__main__':
    app.run()

 * ngrok tunnel "NgrokTunnel: "https://22c5-34-148-141-18.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [03/Sep/2024 17:14:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Sep/2024 17:15:00] "GET /static/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Sep/2024 17:15:01] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
from pyngrok import ngrok

# Terminate open tunnels if any
ngrok.kill()

# Create a new tunnel
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://c6fa-34-148-141-18.ngrok-free.app" -> "http://localhost:5000"
